## Data Preprocessing

#### Import Libraries and Load Dataset

In [17]:
import pandas as pd
import numpy as np

## Load dataset
df = pd.read_csv(r"D:\Hip-Replacement-EffiCare\Efficare Data\hospital_inpatient_discharges_totalhipreplacement.csv")

## Display Data
df.head(3)

,health_service_area,hospital_county,operating_certificate_number,facility_id,facility_name,age_group,zip_code_3_digits,gender,race,ethnicity,...,apr_mdc_code,apr_mdc_description,apr_severity_of_illness_code,apr_severity_of_illness_description,apr_risk_of_mortality,apr_medical_surgical_description,attending_provider_license_number,operating_provider_license_number,total_charges,total_costs
0,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,70 or Older,148,F,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,2,Moderate,Minor,Surgical,213053,213053,35681.75,17400.79
1,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,148,F,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,2,Moderate,Minor,Surgical,213053,213053,34289.25,16657.95
2,Western NY,Allegany,228000,39,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...,50 to 69,147,M,White,Not Span/Hispanic,...,8,Diseases and Disorders of the Musculoskeletal ...,2,Moderate,Minor,Surgical,213053,213053,30436.00,14703.62


#### Basic Info & Missing Values

In [18]:
## dataset shape

# rows
print(df.shape[0])

# columns 
print(df.shape[1])

26594
30


In [4]:
## dataset type and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26594 entries, 0 to 26593
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   health_service_area                  26594 non-null  object 
 1   hospital_county                      26594 non-null  object 
 2   operating_certificate_number         26594 non-null  int64  
 3   facility_id                          26594 non-null  int64  
 4   facility_name                        26594 non-null  object 
 5   age_group                            26594 non-null  object 
 6   zip_code_3_digits                    26574 non-null  object 
 7   gender                               26594 non-null  object 
 8   race                                 26594 non-null  object 
 9   ethnicity                            26594 non-null  object 
 10  length_of_stay                       26594 non-null  int64  
 11  type_of_admission           

In [19]:
## count missing values per column
df.isnull().sum()

health_service_area                     0
hospital_county                         0
operating_certificate_number            0
facility_id                             0
facility_name                           0
age_group                               0
zip_code_3_digits                      20
gender                                  0
race                                    0
ethnicity                               0
length_of_stay                          0
type_of_admission                       0
patient_disposition                     0
discharge_year                          0
ccs_diagnosis_code                      0
ccs_diagnosis_description               0
ccs_procedure_code                      0
ccs_procedure_description               0
apr_drg_code                            0
apr_drg_description                     0
apr_mdc_code                            0
apr_mdc_description                     0
apr_severity_of_illness_code            0
apr_severity_of_illness_descriptio

#### Handling Missing values

In [20]:
## replace missing value in categorical cols with 'Unknown'
categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

In [21]:
## replace missing values in numeric cols with median
numeric_cols = df.select_dtypes(include=['int64','float']).columns

for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

#### Standardizing Categorical cols

In [22]:
## convert categorical cols to lowercase and remove extra space

for col in categorical_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()

#### Outlier Treatment (IQR Capping)

In [24]:
## Applied to : {total_charges, total_costs, length_of_stay}

def iqr_cap(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_cap = Q1 - 1.5 * IQR
    upper_cap = Q3 + 1.5 * IQR

    df[column] = np.where(df[column] < lower_cap, lower_cap, df[column])
    df[column] = np.where(df[column] > upper_cap, upper_cap, df[column])

## applying IQR capping
num_cols_to_cap = ["total_charges", "total_cost", "length_of_stay"]

for col in num_cols_to_cap:
    if col in df.columns:
        iqr_cap(col)

#### Feature Engineering

In [26]:
## Charge-to-cost-ratio
## Efficiency index

# Avoid divide by zero error

df["charge_to_cost_ratio"] = df["total_charges"] / df["total_costs"].replace(0, np.nan)

df["efficiency_index"] = df["length_of_stay"] / df["total_costs"].replace(0, np.nan)

df.head()

,health_service_area,hospital_county,operating_certificate_number,facility_id,facility_name,age_group,zip_code_3_digits,gender,race,ethnicity,...,apr_severity_of_illness_code,apr_severity_of_illness_description,apr_risk_of_mortality,apr_medical_surgical_description,attending_provider_license_number,operating_provider_license_number,total_charges,total_costs,charge_to_cost_ratio,efficiency_index
0,western ny,allegany,228000,39,memorial hosp of wm f & gertrude f jones a/k/a...,70 or older,148,f,white,not span/hispanic,...,2,moderate,minor,surgical,213053,213053,35681.75,17400.79,2.050582,0.000259
1,western ny,allegany,228000,39,memorial hosp of wm f & gertrude f jones a/k/a...,50 to 69,148,f,white,not span/hispanic,...,2,moderate,minor,surgical,213053,213053,34289.25,16657.95,2.058432,0.000240
2,western ny,allegany,228000,39,memorial hosp of wm f & gertrude f jones a/k/a...,50 to 69,147,m,white,not span/hispanic,...,2,moderate,minor,surgical,213053,213053,30436.00,14703.62,2.069966,0.000204
3,western ny,allegany,228000,39,memorial hosp of wm f & gertrude f jones a/k/a...,50 to 69,147,m,white,not span/hispanic,...,1,minor,minor,surgical,213053,213053,28699.00,13903.30,2.064186,0.000216
4,western ny,allegany,228000,39,memorial hosp of wm f & gertrude f jones a/k/a...,50 to 69,148,m,white,not span/hispanic,...,1,minor,minor,surgical,213053,213053,18421.00,8258.43,2.230569,0.000242


#### Encoding categorical variables

In [27]:
## a - label encoding for ordered categpries (apr_severity_of)illbess_description)

severity_map = {
    "minor":1,
    "moderate":2,
    "major":3,
    "extreme":4
}

df["apr_severity_encode"] = df["apr_severity_of_illness_description"].map(severity_map)

In [28]:
## b - One Hot encoding for nominal variables (gender, race, ethinicity, etc)

one_hot_columns = [
    "gender", "race", "ethnicity",
    "facility_name", "hospital_county",
    "health_service_area", "age_group"
]

df = pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

#### final cleaned dataset export

In [29]:
df.to_csv("cleaned_hip_replacement_data.csv",index=False)